In [1]:
import re,sys,os,numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, LSTM, Layer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from time import time,sleep
from tensorflow.keras.optimizers import SGD
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [9]:
#data_folder="train_data"
    
n_data=len(os.listdir('train_data'))
test_size=len(os.listdir('test_data'))

n_of_epochs=20
mini_batch_size=16
#network='cnn'

In [11]:
# Function to convert images to vectors
def get_data(folder):

    labels=[]
    data=[]

    for img_name in folder:
    
        path=data_folder+"/"+img_name
        
        label=img_name[4]
        labels.append(label)
        
        #convert image to vector
        img = load_img(path, target_size=(32, 32))
        img = img.convert('L')
        img_vector = img_to_array(img)/255
        data.append(img_vector)
       
    # convert labels into numeric form
    encoder = LabelEncoder()  
    labels = encoder.fit_transform(labels)
    
    X = np.array(data)
    y = np.array(labels)
    
    if network=='cnn':
        X = X.reshape([-1, 32, 32,1])
    elif network=='lstm':
        X = X.reshape([-1, 32, 32])
    
    return X,y

In [12]:
# cnn implementation
def cnn(x_train,y_train):

    model=Sequential()
    model.add(Conv2D(6, kernel_size=(5, 5), kernel_regularizer='l2',activation='relu',padding='same', input_shape=(32,32,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    model.add(Conv2D(16, kernel_size=(3, 3), kernel_regularizer='l2',activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(120, kernel_size=(3, 3), kernel_regularizer='l2',activation='relu',padding='same'))
      
    model.add(Flatten())
    model.add(Dense(84, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model
        

In [13]:
# lstm implemetation  
def lstm(x,y):

    model=Sequential() 
    model.add(LSTM(32, activation='relu',bias_regularizer='l2', input_shape=(32,32)))
    model.add(Dense(6,activation='softmax'))
    return model   

In [14]:
def train(x,y,network):

    if network=='cnn':
        model=cnn(x,y)
        model_file='model_cnn.h5'
    elif network=='lstm':
        model=lstm(x,y)
        model_file='model_lstm.h5'
        
    # compile the model using 'SGD' optimizer
    opt = SGD (learning_rate=0.01, momentum = 0.9, clipnorm = 1.0)
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
    size=0
    start_train=time()
    
     # Fitting the model for given number of epochs, print the progress on the terminal
    model.fit(x_train,y_train,epochs=n_of_epochs,batch_size=mini_batch_size,verbose=1)
                
    train_time=time()-start_train
    print("\nProcessing complete!")
    print("Total number of items trained on", n_data)
    print("Training Runtime",train_time)
        
    # save the model in .h5 file for testing
    if os.path.exists(model_file)==True:
        os.remove(model_file)
    
    model.save(model_file)

In [21]:
def test(x_test,y_test, model_file):

    if os.path.exists(model_file)==False:
        print("Model file not found")
        exit()
    
    # load the model from the model file
    model=load_model(model_file)
    start = time()
    
    # predict labels for test data using the loaded model
    predictions = model.predict(x_test)
    
    # predicted labels
    y_pred = np.argmax(predictions, axis=1)
    
    # compute accuracy and confusion matrix
    test_loss, test_acc = model.evaluate(x_test,y_test,verbose=1)
    matrix = confusion_matrix(y_test, y_pred)
    
    print("Processing complete!")
    print("Total number of items trained on", n_data)
    print("Total number of items tested on", test_size)    
    print("Accuracy",test_acc*100,"%")
    print("Confusion Matrix\n",matrix)
    print("Testing Runtime",time()-start)    
 


In [17]:

x_train, y_train = get_data(os.listdir("train_data")) 
train(x_train, y_train, 'cnn')


Epoch 1/20
621/621 [==============================] - 11s 15ms/step - loss: 1.7099 - accuracy: 0.3090
Epoch 2/20
621/621 [==============================] - 9s 15ms/step - loss: 1.4766 - accuracy: 0.3704
Epoch 3/20
621/621 [==============================] - 9s 15ms/step - loss: 1.2070 - accuracy: 0.5391
Epoch 4/20
621/621 [==============================] - 9s 15ms/step - loss: 0.8740 - accuracy: 0.7195
Epoch 5/20
621/621 [==============================] - 9s 15ms/step - loss: 0.7241 - accuracy: 0.7994
Epoch 6/20
621/621 [==============================] - 9s 15ms/step - loss: 0.6364 - accuracy: 0.8385
Epoch 7/20
621/621 [==============================] - 9s 15ms/step - loss: 0.5812 - accuracy: 0.8589
Epoch 8/20
621/621 [==============================] - 9s 15ms/step - loss: 0.5487 - accuracy: 0.8739
Epoch 9/20
621/621 [==============================] - 9s 15ms/step - loss: 0.5267 - accuracy: 0.8821
Epoch 10/20
621/621 [==============================] - 9s 15ms/step - loss: 0.5057 - accur

In [22]:

x_test, y_test=get_data(os.listdir("test_data"))
test(x_test, y_test,"model_cnn.h5")

66/66 [==============================] - 1s 11ms/step - loss: 0.3603 - accuracy: 0.9430
Processing complete!
Total number of items trained on 9930
Total number of items tested on 2088
Accuracy 94.30076479911804 %
Confusion Matrix
 [[ 97  21   0   0   2   0]
 [  0 289   7   1   2   1]
 [  0  20 462   1   4   5]
 [  0   0   2 421   1   8]
 [  0   0   1   1 112   0]
 [  0   1   1  39   1 588]]
Testing Runtime 2.235313892364502


In [24]:

x_train, y_train = get_data(os.listdir("train_data")) 
train(x_train, y_train, 'lstm')
x_test, y_test=get_data(os.listdir("test_data"))
test(x_test, y_test,"model_lstm.h5")

Epoch 1/20
621/621 [==============================] - 13s 15ms/step - loss: 1.6786 - accuracy: 0.3054
Epoch 2/20
621/621 [==============================] - 9s 15ms/step - loss: 1.4681 - accuracy: 0.3547
Epoch 3/20
621/621 [==============================] - 9s 15ms/step - loss: 1.4023 - accuracy: 0.4021
Epoch 4/20
621/621 [==============================] - 10s 15ms/step - loss: 1.3193 - accuracy: 0.4624
Epoch 5/20
621/621 [==============================] - 9s 15ms/step - loss: 1.1177 - accuracy: 0.5779
Epoch 6/20
621/621 [==============================] - 10s 15ms/step - loss: 0.9213 - accuracy: 0.6628
Epoch 7/20
621/621 [==============================] - 9s 15ms/step - loss: 0.8298 - accuracy: 0.7092
Epoch 8/20
621/621 [==============================] - 9s 15ms/step - loss: 0.7821 - accuracy: 0.7291
Epoch 9/20
621/621 [==============================] - 9s 15ms/step - loss: 0.6898 - accuracy: 0.7673
Epoch 10/20
621/621 [==============================] - 9s 15ms/step - loss: 0.6476 - acc